In [133]:
# v v v v vv v v v v v v v v v v v vv v v v v v v v v v v v vv v v v v v v v v v v v vv v v v v 
# How do NWS point forecasts (PFM) and grid forecasts measure up: are they similar to averages?
# ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ ^ 

In [134]:
import pandas as pd
import json
import requests
import re
from datetime import datetime
import geopy
import sys
import os
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from datetime import datetime
import numpy as np
import folium
from sklearn.metrics import mean_squared_error
from IPython.display import display_html

In [135]:
import NWSPPFunctions as parsfunc
import forecast_merger as merg
import nws

In [136]:
apistarturl = 'https://api.weather.gov'

In [137]:
PFM = parsfunc.get_api_products(29.9511,-90.0715)

{'productCode': 'PFM', 'productName': 'Point Forecast Matrices'}


In [138]:
product_json = parsfunc.returnthree_hr_timeseries_json_from_pfm(PFM)

In [139]:
dfpfm = pd.DataFrame(product_json)
cor = dfpfm['location'].tolist()
coords = []

for x in range(0,len(cor)):
    c1 = cor[x].replace('N ',',')
    c2 = c1.replace('W E', '')
    c3 = c2.replace(' ','-')
    coords.append(c3)
    
#In this part we give the coordinates the correct format lat, lon to make the API request of the GRID forecasts

df = pd.DataFrame(coords, columns = ['Coordinates'])
df[['lat','lon']]=df.Coordinates.str.split(',',expand=True)

coords = df.to_dict('index')
df

Coordinates    lat     lon
0  31.18,-90.47  31.18  -90.47
1  30.53,-91.15  30.53  -91.15
2  30.25,-89.77  30.25  -89.77
3  30.00,-90.25  30.00  -90.25
4  30.07,-89.93  30.07  -89.93
5  30.42,-89.08  30.42  -89.08
6  30.47,-88.53  30.47  -88.53

In [140]:
m = folium.Map(location=[30.50,-90.0715],zoom_start=8,tiles='Stamen Terrain')

for i in range(0,len(df)):
    folium.Marker(
      location=[df.iloc[i]['lat'], df.iloc[i]['lon']],
      popup=df.iloc[i]['Coordinates'],
   ).add_to(m)
m

In [141]:
#if there's a None value in the list, won't be possible to convert to dataframe. That's the next loop
#tries to get the forecast untill it succeeds

json_forecast = []

for i in range(0,len(coords)):
    
    nwsforecast = None
    
    lat = coords[i]['lat']
    lon = coords[i]['lon']
    
    while nwsforecast == None: 
        nwsforecast = nws.collect_NWS_gridpoint_forecast_from_point(float(lat), float(lon), gridpoints_d=False)
        
        if nwsforecast == None:
            print("It wasn't possible to collect the forecast ", coords[i]['lat'],coords[i]['lat'], "in the position", i)
        else:
            print("The forecast was successfully collected for the coordinates ", coords[i]['lat'],coords[i]['lat'], "in the position", i)

    json_forecast.append(nwsforecast)

json_forecast


It wasn't possible to collect the forecast  31.18 31.18 in the position 0
{'type': 'Polygon', 'coordinates': [[[-90.4467206, 31.2172522], [-90.44761190000001, 31.1945594], [-90.4210723, 31.1937946], [-90.42017580000001, 31.2164872], [-90.4467206, 31.2172522]]]}
bad grid: 51/142
{'type': 'Polygon', 'coordinates': [[[-90.4741402, 31.1953151], [-90.4750256, 31.172629999999998], [-90.44851469999999, 31.1718704], [-90.44762409999998, 31.1945554], [-90.4741402, 31.1953151]]]}
good grid: 50/141
The forecast was successfully collected for the coordinates  31.18 31.18 in the position 0
{'type': 'Polygon', 'coordinates': [[[-91.1328192, 30.5537028], [-91.1335721, 30.5309761], [-91.1071947, 30.5303257], [-91.10643669999999, 30.5530524], [-91.1328192, 30.5537028]]]}
bad grid: 26/112
It wasn't possible to collect the forecast  30.53 30.53 in the position 1
{'type': 'Polygon', 'coordinates': [[[-91.1328192, 30.5537028], [-91.1335721, 30.5309761], [-91.1071947, 30.5303257], [-91.10643669999999, 30.55

[{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
   {'@version': '1.1',
    'wmoUnit': 'https://codes.wmo.int/common/unit/',
    'nwsUnit': 'https://api.weather.gov/ontology/unit/'}],
  'id': 'https://api.weather.gov/gridpoints/LIX/50,141',
  'type': 'Feature',
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-90.4741402, 31.1953151],
     [-90.4750256, 31.172629999999998],
     [-90.44851469999999, 31.1718704],
     [-90.44762409999998, 31.1945554],
     [-90.4741402, 31.1953151]]]},
  'properties': {'@id': 'https://api.weather.gov/gridpoints/LIX/50,141',
   '@type': 'wx:Gridpoint',
   'updateTime': '2022-07-28T22:55:39+00:00',
   'validTimes': '2022-07-28T16:00:00+00:00/P8DT6H',
   'elevation': {'unitCode': 'wmoUnit:m', 'value': 136.8552},
   'forecastOffice': 'https://api.weather.gov/offices/LIX',
   'gridId': 'LIX',
   'gridX': '50',
   'gridY': '141',
   'temperature': {'uom': 'wmoUnit:degC',
    'values': [{'validTime': '2022-07-28T16:00:00+00:00/PT1H

In [143]:
df= pd.json_normalize(json_forecast, max_level=2)
pd.options.display.max_rows = None
forec = df[['id','geometry.coordinates','properties.temperature.values','properties.dewpoint.values','properties.relativeHumidity.values','properties.windSpeed.values']]
forecast = forec.to_dict('index')
forecast[0].keys()
df
#Select only the information that we can use to compare with the PFM

@context  \
0  [https://geojson.org/geojson-ld/geojson-contex...   
1  [https://geojson.org/geojson-ld/geojson-contex...   
2  [https://geojson.org/geojson-ld/geojson-contex...   
3  [https://geojson.org/geojson-ld/geojson-contex...   
4  [https://geojson.org/geojson-ld/geojson-contex...   
5  [https://geojson.org/geojson-ld/geojson-contex...   
6  [https://geojson.org/geojson-ld/geojson-contex...   

                                               id     type geometry.type  \
0   https://api.weather.gov/gridpoints/LIX/50,141  Feature       Polygon   
1   https://api.weather.gov/gridpoints/LIX/25,111  Feature       Polygon   
2   https://api.weather.gov/gridpoints/LIX/78,101  Feature       Polygon   
3    https://api.weather.gov/gridpoints/LIX/60,89  Feature       Polygon   
4    https://api.weather.gov/gridpoints/LIX/72,93  Feature       Polygon   
5  https://api.weather.gov/gridpoints/LIX/103,109  Feature       Polygon   
6  https://api.weather.gov/gridpoints/LIX/124,113  Feature       Polygon   

                                geometry.coordinates  \
0  [[[-90.4741402, 31.1953151], [-90.4750256, 31....   
1  [[[-91.1599535, 30.5316174], [-91.1607007, 30....   
2  [[[-89.7724548, 30.2640782], [-89.7734695, 30....   
3  [[[-90.2571621, 30.0063661], [-90.2580811, 29....   
4  [[[-89.9381828, 30.0874333], [-89.939164, 30.0...   
5  [[[-89.1056655, 30.4225364], [-89.1068114, 30....   
6  [[[-88.5475913, 30.4916451], [-88.548846, 30.4...   

                                   properties.@id properties.@type  \
0   https://api.weather.gov/gridpoints/LIX/50,141     wx:Gridpoint   
1   https://api.weather.gov/gridpoints/LIX/25,111     wx:Gridpoint   
2   https://api.weather.gov/gridpoints/LIX/78,101     wx:Gridpoint   
3    https://api.weather.gov/gridpoints/LIX/60,89     wx:Gridpoint   
4    https://api.weather.gov/gridpoints/LIX/72,93     wx:Gridpoint   
5  https://api.weather.gov/gridpoints/LIX/103,109     wx:Gridpoint   
6  https://api.weather.gov/gridpoints/LIX/124,113     wx:Gridpoint   

       properties.updateTime             properties.validTimes  \
0  2022-07-28T22:55:39+00:00  2022-07-28T16:00:00+00:00/P8DT6H   
1  2022-07-28T22:55:39+00:00  2022-07-28T16:00:00+00:00/P8DT6H   
2  2022-07-28T22:55:39+00:00  2022-07-28T16:00:00+00:00/P8DT6H   
3  2022-07-28T22:55:39+00:00  2022-07-28T16:00:00+00:00/P8DT6H   
4  2022-07-28T22:55:39+00:00  2022-07-28T16:00:00+00:00/P8DT6H   
5  2022-07-28T22:55:39+00:00  2022-07-28T16:00:00+00:00/P8DT6H   
6  2022-07-28T22:55:39+00:00  2022-07-28T16:00:00+00:00/P8DT6H   

  properties.elevation.unitCode  ...  \
0                     wmoUnit:m  ...   
1                     wmoUnit:m  ...   
2                     wmoUnit:m  ...   
3                     wmoUnit:m  ...   
4                     wmoUnit:m  ...   
5                     wmoUnit:m  ...   
6                     wmoUnit:m  ...   

   properties.potentialOf40mphWindGusts.values  \
0                                           []   
1                                           []   
2                                           []   
3                                           []   
4                                           []   
5                                           []   
6                                           []   

  properties.potentialOf50mphWindGusts.values  \
0                                          []   
1                                          []   
2                                          []   
3                                          []   
4                                          []   
5                                          []   
6                                          []   

  properties.potentialOf60mphWindGusts.values  \
0                                          []   
1                                          []   
2                                          []   
3                                          []   
4                                          []   
5                                     

In [162]:
import geopandas
from shapely import wkt

df.geometry.coordinates = df.geometry.coordinates.apply(load_valid)
gdf = geopandas.GeoDataFrame(df.dropna(), geometry='geometry.coordinates')

folium.GeoJson(data=gdf).add_to(m)
m


AttributeError: 'DataFrame' object has no attribute 'geometry'

In [10]:
def CtoF(temp):
    return ((9/5) * temp) + 32

def blank(value):
    return value

In [11]:
temp = nws.adjust_compressed_nws_timeseries_to_aligned_hourly_timestamped_timeseries(forecast[0]['properties.temperature.values'], value_adj= CtoF, alignment_time=0)
dewp = nws.adjust_compressed_nws_timeseries_to_aligned_hourly_timestamped_timeseries(forecast[0]['properties.dewpoint.values'], value_adj= CtoF, alignment_time=0)
RH = nws.adjust_compressed_nws_timeseries_to_aligned_hourly_timestamped_timeseries(forecast[0]['properties.relativeHumidity.values'], value_adj= blank, alignment_time=0)
winds = nws.adjust_compressed_nws_timeseries_to_aligned_hourly_timestamped_timeseries(forecast[0]['properties.windSpeed.values'], value_adj= blank, alignment_time=0)

In [12]:
dftemp = pd.json_normalize(temp, max_level=1)
dfdew = pd.json_normalize(dewp, max_level=1)
dfrh = pd.json_normalize(RH, max_level=1)
dfws = pd.json_normalize(winds, max_level=1)

In [13]:
forecastgridp = pd.merge(dftemp,dfdew, on='validTime',suffixes=('_temp', '_dewp'))
forecastgridp1 = pd.merge(forecastgridp, dfrh, on='validTime',suffixes=('_1', '_rh'))
forecastgridp2 = pd.merge(forecastgridp1,dfws, on='validTime')
forecastgridp2
gridpoint = forecastgridp2.set_axis(['ValidTime', 'Temp', 'DewP', 'RH','WindSp'], axis=1)
gridpoint
# Here we have the hourly grid forecast for 8 days and 6 hours

ValidTime  Temp  DewP   RH  WindSp
0    1659031200  77.0  73.0   87   0.000
1    1659034800  81.0  74.0   79   5.556
2    1659038400  83.0  74.0   74   9.260
3    1659042000  86.0  73.0   65   7.408
4    1659045600  87.0  73.0   63   7.408
5    1659049200  88.0  73.0   61  11.112
6    1659052800  89.0  72.0   57  11.112
7    1659056400  89.0  72.0   57  14.816
8    1659060000  89.0  72.0   57  14.816
9    1659063600  87.0  72.0   61  14.816
10   1659067200  85.0  73.0   67  14.816
11   1659070800  85.0  72.0   65  12.964
12   1659074400  81.0  73.0   77   9.260
13   1659078000  79.0  73.0   82   5.556
14   1659081600  77.0  72.0   85   5.556
15   1659085200  76.0  73.0   90   5.556
16   1659088800  76.0  73.0   90   5.556
17   1659092400  75.0  73.0   94   3.704
18   1659096000  75.0  72.0   90   5.556
19   1659099600  74.0  72.0   93   5.556
20   1659103200  74.0  72.0   93   3.704
21   1659106800  73.0  72.0   97   3.704
22   1659110400  73.0  72.0   97   5.556
23   1659114000  73.0  72.0   97   1.852
24   1659117600  78.0  74.0   88   7.408
25   1659121200  83.0  75.0   77  11.112
26   1659124800  85.0  74.0   70  11.112
27   1659128400  88.0  74.0   63  11.112
28   1659132000  90.0  73.0   58  11.112
29   1659135600  92.0  73.0   54  11.112
30   1659139200  92.0  73.0   54  11.112
31   1659142800  91.0  73.0   56  14.816
32   1659146400  91.0  73.0   56  14.816
33   1659150000  89.0  73.0   59  14.816
34   1659153600  86.0  73.0   65  12.964
35   1659157200  84.0  73.0   70  11.112
36   1659160800  82.0  74.0   77   7.408
37   1659164400  78.0  74.0   88   5.556
38   1659168000  77.0  73.0   87   5.556
39   1659171600  76.0  73.0   90   5.556
40   1659175200  76.0  73.0   90   5.556
41   1659178800  76.0  72.0   87   5.556
42   1659182400  76.0  72.0   87   5.556
43   1659186000  76.0  72.0   87   3.704
44   1659189600  75.0  72.0   90   3.704
45   1659193200  73.0  72.0   97   1.852
46   1659196800  72.0  72.0  100   1.852
47   1659200400  73.0  72.0   97   1.852
48   1659204000  77.0  73.0   87   3.704
49   1659207600  81.0  73.0   77   7.408
50   1659211200  85.0  74.0   70  11.112
51   1659214800  89.0  74.0   61  11.112
52   1659218400  90.0  73.0   58  11.112
53   1659222000  92.0  73.0   54  11.112
54   1659225600  92.0  73.0   54  11.112
55   1659229200  92.0  73.0   54  11.112
56   1659232800  92.0  73.0   54  11.112
57   1659236400  90.0  73.0   58  11.112
58   1659240000  88.0  73.0   61   9.260
59   1659243600  85.0  73.0   67   9.260
60   1659247200  82.0  73.0   74   7.408
61   1659250800  79.0  72.0   79   7.408
62   1659254400  77.0  72.0   85   5.556
63   1659258000  76.0  72.0   87   5.556
64   1659261600  76.0  72.0   87   5.556
65   1659265200  76.0  72.0   87   5.556
66   1659268800  75.0  72.0   90   3.704
67   1659272400  74.0  71.0   90   3.704
68   1659276000  73.0  71.0   93   1.852
69   1659279600  72.0  71.0   97   0.000
70   1659283200  72.0  71.0   97   0.000
71   1659286800  73.0  71.0   93   0.000
72   1659290400  77.0  72.0   85   3.704
73   1659294000  81.0  73.0   77   7.408
74   1659297600  86.0  74.0   68  11.112
75   1659301200  89.0  74.0   61  11.112
76   1659304800  91.0  73.0   56  11.112
77   1659308400  93.0  72.0   51  11.112
78   1659312000  94.0  71.0   47  11.112
79   1659315600  94.0  71.0   47  12.964
80   1659319200  93.0  71.0   49  12.964
81   1659322800  91.0  71.0   52  12.964
82   1659326400  89.0  72.0   57  12.964
83   1659330000  86.0  72.0   63  11.112
84   1659333600  83.0  72.0   70   9.260
85   1659337200  80.0  72.0   77   9.260
86   1659340800  78.0  72.0   82   7.408
87   1659344400  76.0  72.0   87   5.556
88   1659348000  76.0  72.0   87   5.556
89   1659351600  75.0  72.0   90   3.704
90   1659355200  75.0  72.0   90   3.704
91   1659358800  75.0  72.0   90   3.704
92   1659362400  74.0  72.0   93   3.704
93   1659366000  73.0  72.0   97   1.852
94   1659369600  73.0  72.0   97   1.852
95   1659373200  74.0  72.0   93   1.852
96   1659376800  77.0  73.0  

In [102]:
pfm = pd.json_normalize(product_json, max_level = 2)
pfm2 = pfm['timeseries']
# iloc extracts the rows of the timeseries to create a new dataframe for each timeseries
rows = pfm2.iloc[0]
pfmdf = pd.DataFrame(rows)
pfmdf = pfmdf.set_axis(['hours', 'temp', 'dewP', 'RH','windD','WindSp','clouds'], axis=1)
del pfmdf['windD']
del pfmdf['clouds']
pfmd = pfmdf.replace({'   ': '0'})
pfmd['Temp'] = pd.to_numeric(pfmd['temp'], downcast="float")
pfmd['DewP'] = pd.to_numeric(pfmd['dewP'], downcast="float")
pfmd['RH'] = pd.to_numeric(pfmd['RH'], downcast="float")
pfmd['WindSp'] = pd.to_numeric(pfmd['WindSp'], downcast="float")


In [103]:
pfmd = pd.DataFrame(np.repeat(pfmd.values, 3, axis=0), columns=pfmd.columns)

def createList(r1, r2):
    return [item for item in range(r1, r2)]

r1, r2 = int(pfmd['hours'][0]),(len(pfmd['hours'])+int(pfmd['hours'][0]))
                                 
hours = createList(r1, r2)
pfmd['hours'] = hours

pfmd 

hours temp dewP RH WindSp Temp DewP
0       4    0    0  0      0    0    0
1       5    0    0  0      0    0    0
2       6    0    0  0      0    0    0
3       7   73  71  93      0   73   71
4       8   73  71  93      0   73   71
5       9   73  71  93      0   73   71
6      10   83  74  74      6   83   74
7      11   83  74  74      6   83   74
8      12   83  74  74      6   83   74
9      13   88  73  61      6   88   73
10     14   88  73  61      6   88   73
11     15   88  73  61      6   88   73
12     16   89  72  57      9   89   72
13     17   89  72  57      9   89   72
14     18   89  72  57      9   89   72
15     19   83  73  72      8   83   73
16     20   83  73  72      8   83   73
17     21   83  73  72      8   83   73
18     22   77  73  87      3   77   73
19     23   77  73  87      3   77   73
20     24   77  73  87      3   77   73
21     25   75  72  90      2   75   72
22     26   75  72  90      2   75   72
23     27   75  72  90      2   75   72
24     28   74  72  93      2   74   72
25     29   74  72  93      2   74   72
26     30   74  72  93      2   74   72
27     31   73  72  97      2   73   72
28     32   73  72  97      2   73   72
29     33   73  72  97      2   73   72
30     34   84  74  72      6   84   74
31     35   84  74  72      6   84   74
32     36   84  74  72      6   84   74
33     37   89  73  59      6   89   73
34     38   89  73  59      6   89   73
35     39   89  73  59      6   89   73
36     40   88  73  61      8   88   73
37     41   88  73  61      8   88   73
38     42   88  73  61      8   88   73
39     43   82  73  74      6   82   73
40     44   82  73  74      6   82   73
41     45   82  73  74      6   82   73
42     46   77  73  87      2   77   73
43     47   77  73  87      2   77   73
44     48   77  73  87      2   77   73
45     49   75  72  90      3   75   72
46     50   75  72  90      3   75   72
47     51   75  72  90      3   75   72
48     52   74  72  93      3   74   72
49     53   74  72  93      3   74   72
50     54   74  72  93      3   74   72
51     55   73  72  97      2   73   72
52     56   73  72  97      2   73   72
53     57   73  72  97      2   73   72
54     58   83  75  77      5   83   75
55     59   83  75  77      5   83   75
56     60   83  75  77      5   83   75
57     61   90  73  58      6   90   73
58     62   90  73  58      6   90   73
59     63   90  73  58      6   90   73
60     64   89  73  59      6   89   73
61     65   89  73  59      6   89   73
62     66   89  73  59      6   89   73
63     67   84   73 70      5   84   73
64     68   84   73 70      5   84   73
65     69   84   73 70      5   84   73

In [104]:
sd = pfm['start_date'].tolist()

dates = []

for x in range(0,len(sd)):
    c1 = sd[x].replace('Thu ','')
    c2 = c1.replace('22 ', '2022')
    dates.append(c2)

date_string = dates[0]
pfmd['start_time'] = date_string
pfmd['ValidTime'] = datetime.strptime(pfmd['start_time'][0], "%m/%d/%Y").timestamp()
pfmd['hts'] = pfmd['hours'].multiply(3600)
pfmd['ValidTime']= pfmd['ValidTime'] + pfmd['hts']
del pfmd['hts']
del pfmd['start_time']
del pfmd['hours']

pfmd = pfmd.reindex(columns=['ValidTime','temp','dewP','RH','WindSp'])


In [106]:
pd.set_option('display.float_format', '{:.0f}'.format)
pfmd = pfmd.set_axis(['ValidTime', 'Temp', 'DewP', 'RH','WindSp'], axis=1)

ValidTime Temp DewP RH WindSp
0  1658998800    0    0  0      0
1  1659002400    0    0  0      0
2  1659006000    0    0  0      0
3  1659009600   73  71  93      0
4  1659013200   73  71  93      0
5  1659016800   73  71  93      0
6  1659020400   83  74  74      6
7  1659024000   83  74  74      6
8  1659027600   83  74  74      6
9  1659031200   88  73  61      6
10 1659034800   88  73  61      6
11 1659038400   88  73  61      6
12 1659042000   89  72  57      9
13 1659045600   89  72  57      9
14 1659049200   89  72  57      9
15 1659052800   83  73  72      8
16 1659056400   83  73  72      8
17 1659060000   83  73  72      8
18 1659063600   77  73  87      3
19 1659067200   77  73  87      3
20 1659070800   77  73  87      3
21 1659074400   75  72  90      2
22 1659078000   75  72  90      2
23 1659081600   75  72  90      2
24 1659085200   74  72  93      2
25 1659088800   74  72  93      2
26 1659092400   74  72  93      2
27 1659096000   73  72  97      2
28 1659099600   73  72  97      2
29 1659103200   73  72  97      2
30 1659106800   84  74  72      6
31 1659110400   84  74  72      6
32 1659114000   84  74  72      6
33 1659117600   89  73  59      6
34 1659121200   89  73  59      6
35 1659124800   89  73  59      6
36 1659128400   88  73  61      8
37 1659132000   88  73  61      8
38 1659135600   88  73  61      8
39 1659139200   82  73  74      6
40 1659142800   82  73  74      6
41 1659146400   82  73  74      6
42 1659150000   77  73  87      2
43 1659153600   77  73  87      2
44 1659157200   77  73  87      2
45 1659160800   75  72  90      3
46 1659164400   75  72  90      3
47 1659168000   75  72  90      3
48 1659171600   74  72  93      3
49 1659175200   74  72  93      3
50 1659178800   74  72  93      3
51 1659182400   73  72  97      2
52 1659186000   73  72  97      2
53 1659189600   73  72  97      2
54 1659193200   83  75  77      5
55 1659196800   83  75  77      5
56 1659200400   83  75  77      5
57 1659204000   90  73  58      6
58 1659207600   90  73  58      6
59 1659211200   90  73  58      6
60 1659214800   89  73  59      6
61 1659218400   89  73  59      6
62 1659222000   89  73  59      6
63 1659225600   84   73 70      5
64 1659229200   84   73 70      5
65 1659232800   84   73 70      5

In [112]:
pfmd = pfmd.drop([0,1,2,3,4,5,6,7,8],axis=0)

In [124]:
gridpoint = gridpoint.head(len(pfmd))
df1_styler = gridpoint.style.set_table_attributes("style='display:inline'").set_caption('Grid Point Forecast')
df2_styler = pfmd.style.set_table_attributes("style='display:inline'").set_caption('Point Forecast Matrix')
display_html(df1_styler._repr_html_()+df2_styler._repr_html_(), raw=True)

Grid Point Forecast 
 
 
   
 ValidTime 
 Temp 
 DewP 
 RH 
 WindSp 
 
 
 
 
 0 
 1659031200 
 77.000000 
 73.000000 
 87 
 0.000000 
 
 
 1 
 1659034800 
 81.000000 
 74.000000 
 79 
 5.556000 
 
 
 2 
 1659038400 
 83.000000 
 74.000000 
 74 
 9.260000 
 
 
 3 
 1659042000 
 86.000000 
 73.000000 
 65 
 7.408000 
 
 
 4 
 1659045600 
 87.000000 
 73.000000 
 63 
 7.408000 
 
 
 5 
 1659049200 
 88.000000 
 73.000000 
 61 
 11.112000 
 
 
 6 
 1659052800 
 89.000000 
 72.000000 
 57 
 11.112000 
 
 
 7 
 1659056400 
 89.000000 
 72.000000 
 57 
 14.816000 
 
 
 8 
 1659060000 
 89.000000 
 72.000000 
 57 
 14.816000 
 
 
 9 
 1659063600 
 87.000000 
 72.000000 
 61 
 14.816000 
 
 
 10 
 1659067200 
 85.000000 
 73.000000 
 67 
 14.816000 
 
 
 11 
 1659070800 
 85.000000 
 72.000000 
 65 
 12.964000 
 
 
 12 
 1659074400 
 81.000000 
 73.000000 
 77 
 9.260000 
 
 
 13 
 1659078000 
 79.000000 
 73.000000 
 82 
 5.556000 
 
 
 14 
 1659081600 
 77.000000 
 72.000000 
 85 
 5.556000 
 
 
 15 
 1659085200 
 76.000000 
 73.000000 
 90 
 5.556000 
 
 
 16 
 1659088800 
 76.000000 
 73.000000 
 90 
 5.556000 
 
 
 17 
 1659092400 
 75.000000 
 73.000000 
 94 
 3.704000 
 
 
 18 
 1659096000 
 75.000000 
 72.000000 
 90 
 5.556000 
 
 
 19 
 1659099600 
 74.000000 
 72.000000 
 93 
 5.556000 
 
 
 20 
 1659103200 
 74.000000 
 72.000000 
 93 
 3.704000 
 
 
 21 
 1659106800 
 73.000000 
 72.000000 
 97 
 3.704000 
 
 
 22 
 1659110400 
 73.000000 
 72.000000 
 97 
 5.556000 
 
 
 23 
 1659114000 
 73.000000 
 72.000000 
 97 
 1.852000 
 
 
 24 
 1659117600 
 78.000000 
 74.000000 
 88 
 7.408000 
 
 
 25 
 1659121200 
 83.000000 
 75.000000 
 77 
 11.112000 
 
 
 26 
 1659124800 
 85.000000 
 74.000000 
 70 
 11.112000 
 
 
 27 
 1659128400 
 88.000000 
 74.000000 
 63 
 11.112000 
 
 
 28 
 1659132000 
 90.000000 
 73.000000 
 58 
 11.112000 
 
 
 29 
 1659135600 
 92.000000 
 73.000000 
 54 
 11.112000 
 
 
 30 
 1659139200 
 92.000000 
 73.000000 
 54 
 11.112000 
 
 
 31 
 1659142800 
 91.000000 
 73.000000 
 56 
 14.816000 
 
 
 32 
 1659146400 
 91.000000 
 73.000000 
 56 
 14.816000 
 
 
 33 
 1659150000 
 89.000000 
 73.000000 
 59 
 14.816000 
 
 
 34 
 1659153600 
 86.000000 
 73.000000 
 65 
 12.964000 
 
 
 35 
 1659157200 
 84.000000 
 73.000000 
 70 
 11.112000 
 
 
 36 
 1659160800 
 82.000000 
 74.000000 
 77 
 7.408000 
 
 
 37 
 1659164400 
 78.000000 
 74.000000 
 88 
 5.556000 
 
 
 38 
 1659168000 
 77.000000 
 73.000000 
 87 
 5.556000 
 
 
 39 
 1659171600 
 76.000000 
 73.000000 
 90 
 5.556000 
 
 
 40 
 1659175200 
 76.000000 
 73.000000 
 90 
 5.556000 
 
 
 41 
 1659178800 
 76.000000 
 72.000000 
 87 
 5.556000 
 
 
 42 
 1659182400 
 76.000000 
 72.000000 
 87 
 5.556000 
 
 
 43 
 1659186000 
 76.000000 
 72.000000 
 87 
 3.704000 
 
 
 44 
 1659189600 
 75.000000 
 72.000000 
 90 
 3.704000 
 
 
 45 
 1659193200 
 73.000000 
 72.000000 
 97 
 1.852000 
 
 
 46 
 1659196800 
 72.000000 
 72.000000 
 100 
 1.852000 
 
 
 47 
 1659200400 
 73.000000 
 72.000000 
 97 
 1.852000 
 
 
 48 
 1659204000 
 77.000000 
 73.000000 
 87 
 3.704000 
 
 
 49 
 1659207600 
 81.000000 
 73.000000 
 77 
 7.408000 
 
 
 50 
 1659211200 
 85.000000 
 74.000000 
 70 
 11.112000 
 
 
 51 
 1659214800 
 89.000000 
 74.000000 
 61 
 11.112000 
 
 
 52 
 1659218400 
 90.000000 
 73.000000 
 58 
 11.112000 
 
 
 53 
 1659222000 
 92.000000 
 73.000000 
 54 
 11.112000 
 
 
 54 
 1659225600 
 92.000000 
 73.000000 
 54 
 11.112000 
 
 
 55 
 1659229200 
 92.000000 
 73.000000 
 54 
 11.112000 
 
 
 56 
 1659232800 
 92.000000 
 73.000000 
 54 
 11.112000 
 
 
 

 
 Point Forecast Matrix 
 
 
   
 ValidTime 
 Temp 
 DewP 
 RH 
 WindSp 
 
 
 
 
 9 
 1659031200.000000 
 88 
 73 
 61.000000 
 6.000000 
 
 
 10 
 1659034800.000000 
 88 
 73 
 61.000000 
 6.000000 
 
 
 11 
 1659038400.000000 
 88 
 73 
 61.000000 
 6.000000 
 
 
 12 
 1659042000.000000 
 89 
 72 
 57.000000 
 9.000000 
 
 
 13 
 1659045600.000000 
 89 
 72 
 57.00

In [119]:
mean_squared_error(pfmd['Temp'], gridpoint['Temp'])

45.64912280701754

In [120]:
mean_squared_error(pfmd['DewP'], gridpoint['DewP'])

1.2982456140350878

In [121]:
mean_squared_error(pfmd['RH'], gridpoint['RH'])

239.71929824561403

In [123]:
mean_squared_error(pfmd['WindSp'], gridpoint['WindSp'])

28.965421473684213